In [390]:
#import packages
import sys
import pandas as pd
import numpy as np
import TSLPy3 as ts
import GetData as g
# Wind API
from WindPy import w
#Tinysoft API
import TSLPy3 as ts
import matplotlib.pyplot as plt 
import talib
#解决作图中文乱码问题
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
mpl.rcParams['axes.unicode_minus'] = False
import os
from collections import OrderedDict

In [888]:
import sklearn as sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [1247]:
# import xgboost as xgb
import sklearn as sklearn
from sklearn.metrics import r2_score
import statsmodels.api as sm
import xgboost as xgb

In [391]:
#启动Wind API
w.start()
w.isconnected()

True

## 新能源股票池 - new energy industry stock pool

In [340]:
#无法获得因子数据，故删除
stock_delete = ["SH688097","SH688499","SH688660"]

In [341]:
def StockIdWindtoTS(StockId):
    return StockId.map(lambda x:str(x)[-2:]+str(x)[:-3]) 

In [342]:
XNY_stock_pool=pd.read_excel("P:\Documents\量化\行业内因子选股\新能源\stock_pool\新能源_股票list_0715.xlsx",dtype = object)

In [343]:
var_list = ['代码', '名称','上游原材料', '中游设备电池', '中游材料', '中游零部件','下游整车', '下游TMT类', '下游汽车电子', '光伏主产业链', '光伏辅材', '风电', '储能']
XNY_stock=(XNY_stock_pool.groupby(["代码","名称"]).sum()).reset_index()[var_list]
sub_ind_list = ['上游原材料', '中游设备电池', '中游材料', '中游零部件','下游整车', '下游TMT类', '下游汽车电子', '光伏主产业链', '光伏辅材', '风电', '储能']
XNY_stock[sub_ind_list] = (XNY_stock[sub_ind_list]>0)*1

In [344]:
XNY_stock.rename(columns={"代码":"stock_id"},inplace=True)
XNY_stock["stock_id"]=StockIdWindtoTS(XNY_stock["stock_id"])
XNY_stock=XNY_stock[-XNY_stock["stock_id"].isin(stock_delete)]

In [345]:
industry_flag=XNY_stock.drop(["名称"],axis=1)

In [43]:
# XNY_stock.to_excel("P:\Documents\量化\行业内因子选股\新能源\stock_pool\新能源股票_去重.xlsx",index=True)

In [39]:
# stock_output = pd.merge(XNY_stock_pool.iloc[:,:4],XNY_stock,how='left',on=['代码',"名称"]) 

In [393]:
##step1：获取回测区间每周最后一个交易日
# def get_trade_date(start_date, end_date, period='D'):
#     data = w.tdays(start_date, end_date, period=period) #获取每月最后一个交易日
#     trade_dates = data.Data[0]
#     trade_dates = [dt.strftime("%Y-%m-%d") for dt in trade_dates]
#     return trade_dates

# ##设置回测区间
# start_date='20140102'
# end_date='20220718'
# dates=get_trade_date(start_date, end_date, period='D')

In [744]:
start_date='2014-01-02'
end_date='2022-07-18'
dates=g.GetTradeDates(begT=start_date,endT=end_date,type_id='W')
dates = pd.Series(dates).apply(lambda x:str(x)[0:4]+"-"+str(x)[4:6]+"-"+str(x)[6:8]).tolist()

In [773]:
d_temp = pd.DataFrame(dates,columns=["date"])
d_temp["month"]= d_temp["date"].map(lambda x:str(x)[:7])
model_update_date = d_temp.groupby("month")["date"].min().tolist()

In [694]:
#Step2: 构建所有交易日股票池列表
#需要满足以下条件：
# 1.非ST/PT股票
# 2. 剔除上市不满一年的股票
# 3. 剔除选股日停牌而无法买入的股票

In [695]:
origin_loc_ST = "\\\\172.16.2.3\\winfile\\量化投资部\\xunsun\\Database\\StockST\\"
origin_loc_Suspended = "\\\\172.16.2.3\\winfile\\量化投资部\\xunsun\\Database\\StockSuspended\\"

In [696]:
def stock_ST_filter(stock_list,dt):
    stock_list = pd.DataFrame(stock_list,columns=["stock_id"])
    st_list= pd.read_csv(origin_loc_ST+str(dt).replace("-","")+".csv",dtype="object")
    stock_list = stock_list[-stock_list["stock_id"].isin(st_list.stock_id)]
    return stock_list.stock_id.tolist()

In [697]:
def stock_Suspended_filter(stock_list,dt):
    stock_list = pd.DataFrame(stock_list,columns=["stock_id"])
    Suspended_list= pd.read_csv(origin_loc_Suspended+str(dt).replace("-","")+".csv",dtype="object")
    stock_list = stock_list[-stock_list["stock_id"].isin(Suspended_list.stock_id)]
    return stock_list.stock_id.tolist()

### 获取因子数据

In [701]:
fields=["GROWTH_EarningRate_SQIncre",
"GROWTH_NetProfit_SQIncre_2Mcap",
"GROWTH_EstNetProfit_SQIncre_2Mcap",
"GROWTH_NPIncreQ2Q_2Mcap",
"GROWTH_NetProfit_SQYOY",
"GROWTH_NetProfit_Standard",
"GROWTH_ROE_SQYOY",
"GROWTH_ROE_TTMIncre",
"GROWTH_RROC",
"GROWTH_Acceleration",
"GROWTH_SalesEarning_SQYOY",
"GROWTH_Sales_SQYOY",
"GROWTH_Sales_SQIncre_2Mcap",
"GROWTH_SUE",
"GROWTH_DepositReceived_YOYIncre2Debt",
"GROWTH_Salary_YOYIncre2Ast",
"GROWTH_CON_NPCGRATE_1W",
"GROWTH_CON_NPCGRATE_4W",
"GROWTH_CON_NPCGRATE_13W",
"PROFIT_CashRateOfSales_TTM",
"PROFIT_FCFF_5YHL",
"PROFIT_FCFF_5YHL-Std",
"PROFIT_FCFF_Stability",
"PROFIT_GrossProfitMargin_TTM",
"PROFIT_NetProfitMargin_TTM",
"PROFIT_ROE_TTM",
"PROFIT_ROE_SQ",
"PROFIT_ROE_Stability",
"PROFIT_ROIC_SQ",
"PROFIT_SalesEarningRatio_TTM",
"PROFIT_CON_NP_ROLLING",
"PROFIT_CON_OR_ROLLING",
"PROFIT_CON_NP",
"QUALITY_PledgeRatio",
"QUALITY_PublishDateGap",
"QUALITY_LiabilityRatio",
"QUALITY_CFO2Debt",
"QUALITY_CFO2Sales",
"QUALITY_CFO2NP",
"QUALITY_LiabilityRatio_YOYIncre",
"SIZE_LnMarketValue",
"QFII_HoldingPercent",
"QFII_PercentRise_15D",
"QFII_PercentRise_30D",
"QFII_PercentRise_60D",
"LIQUIDITY_ILLIQ",
"LIQUIDITY_ILLIQ_10D",
"LIQUIDITY_TrunoverRate_1Mto3M",
"LIQUIDITY_TurnoverRate_Avg1M",
"LIQUIDITY_Volume_Std1M",
"REVERSAL_StockZF_10D",
"REVERSAL_StockZF_20D",
"REVERSAL_StockZF_40D",
"REVERSAL_StockZF_60D",
"REVERSAL_Extreme_Return",
"REVERSAL_MHigh_20D",
"REVERSAL_M_20D",
"VOLATILITY_RealizedVol_20D",
"VOLATILITY_DASTD",
"VOLATILITY_ATR20",
"VOLATILITY_M_20D",
"VOLATILITY_Amplitude_Mean",
"TECHNICAL_APB_1M",
"TECHNICAL_ARRP",
"TECHNICAL_Alpha_IR",
"TECHNICAL_Amplitude_Diff",
"TECHNICAL_Buy_to_Sell",
"TECHNICAL_BigOrderRatio_20D",
"TECHNICAL_BigOrderRatio_10D",
"TECHNICAL_BigOrderStrength_20D",
"TECHNICAL_BigOrderStrength_10D",
"TECHNICAL_DUVOL",
"TECHNICAL_DownwardVol",
"TECHNICAL_IVR",
"TECHNICAL_PVCorr_20D",
"TECHNICAL_Ratative_TurnVol",
"TECHNICAL_LongTermMom",
"TECHNICAL_Mild_Momentum",
"TECHNICAL_Quantile_Momentum",
"TECHNICAL_Quantile_TurnoverRate",
"TECHNICAL_QFII_MidTerm",
"TECHNICAL_Smart_Money",
"EMOTION_MktConfidence_25D",
"EMOTION_MktConfidence_75D"
]

In [702]:
col_size=["SIZE_LnMarketValue"]
columns_index = ['date',"stock_id"]
columns_industry=list(industry_flag.columns[1:])

In [703]:
factors_quant=gd.GetFactorData(stocks=stock_list,begT=dt.replace("-",""), endT=0, fields=fields,type_id='')
data_factors=pd.merge(factors_quant,industry_flag,on="stock_id",how="left")

In [365]:
data_factors.shape

(457, 97)

### Outliers

In [363]:
#构建绝对中位数处理法函数
def extreme_process_MAD(data,num=3,columns_exclude=['date',"stock_id"]):#data为输入的数据集，如果数值超过num个判断标准则使其等于num个标准
    data_=data.copy() # 为不破坏原始数据，先对其进行拷贝
    feature_names = [i for i in data_.columns.tolist() if i not in columns_exclude] #获取数据集中需测试的因子名
    median=data_[feature_names].median(axis=0) #获取中位数
    MAD=abs(data_[feature_names].sub(median,axis=1)).median(axis=0) #按列索引匹配，并在行中广播
    data_.loc[:,feature_names]=data_.loc[:,feature_names].clip(lower=median-num*1.4826*MAD,upper=median+num*1.4826*MAD,axis=1) #利用clip()函数，将因子取值限定在上下限范围内，即用上下限来代替异常值
    return data_

### Missing Data

In [367]:
#step2:若仍存在缺失值，则用该天的股票池因子的平均值代替
def null_process(data,columns_exclude=['date',"stock_id"]):
    if True in data.isnull().any().values: #判断数据是否存在缺失值
        col=[i for i in data.columns.tolist() if i not in columns_exclude] #得到因子名称
        mean=data[col].mean(axis=0) #按列求均值
        dict_=OrderedDict((key,value) for key,value in zip(mean.index.tolist(),mean.values))
        data.fillna(dict_,inplace=True) #用该天的股票池的平均值来代替
    return data

In [1032]:
def null_process2(data):
    if True in data.isnull().any().values: #判断数据是否存在缺失值
        col=[i for i in data.columns.tolist() if i not in columns_industry+["return"]] #得到因子名称
        mean=data[col].mean(axis=0) #按列求均值
        dict_=OrderedDict((key,value) for key,value in zip(mean.index.tolist(),mean.values))
        data.fillna(dict_,inplace=True) #用该天的股票池的平均值来代替
    return data

### 市值和行业中性化 - Neutralize

In [215]:
#columns_index
#columns_industry
col_size=["SIZE_LnMarketValue"]

In [370]:
import numpy.linalg as la #用来做线性代数运算
#行业市值中性化处理
def data_scale_neutral(data,columns_index,columns_industry,col_size):
    data_=data.copy()
    feature_names = [i for i in data_.columns.tolist() if i not in columns_index+columns_industry+col_size] #获取数据集中需测试的因子名
    n = len(columns_industry)
    x=data_[columns_industry[1:]+col_size] #将行业变量与对数市值合并
    X= np.array(x) #行业虚拟变量与对数市值作为为自变量
    for name in feature_names:
        y = np.array(data_[name])
        if la.matrix_rank(X.T.dot(X)) == (n): #当矩阵满秩时，估计回归参数
            beta_ols = la.inv(X.T.dot(X)).dot(X.T).dot(y)
            residual = y - X.dot(beta_ols) #计算残差，并将其作为剔除行业影响的因子值
        else:
            residual = y #如果逆不存在的话 则 用原值
        data_[name]=residual
    return data_

### 标准化处理 - Standadization

In [375]:
def data_scale_Z_Score(data,columns_exclude=['date',"stock_id"]):
    data_=data.copy() # 为不破坏原始数据，先对其进行拷贝
    feature_names = [i for i in data_.columns.tolist() if i not in columns_exclude] #获取数据集中需测试的因子名
    data_.loc[:,feature_names] = (data_.loc[:,feature_names] - data_.loc[:,feature_names].mean())/data_.loc[:,feature_names].std()
    return data_

In [ ]:
# XNY_stock
#industry_flag=XNY_stock.drop(["名称"],axis=1)

In [1306]:
# df_train=pd.DataFrame()
# for i in range(len(dates)-1):
#     dt = dates[i]
#     if i==(len(dates)-1):
#         dt1 ="2030-1-19"
#     else:
#         dt1 = dates[i+1]
#     stock_list = g.stock_issueDate_filter(XNY_stock.stock_id,dt,delta=360)
#     stock_list = stock_ST_filter(stock_list,dt)
#     stock_list = stock_Suspended_filter(stock_list,dt)
#     factors_quant=gd.GetFactorData(stocks=stock_list,begT=dt.replace("-",""), endT=0, fields=fields,type_id='')
#     data_factors=pd.merge(factors_quant,industry_flag,on="stock_id",how="left")
#     #异常值处理
#     data_factors_1 = extreme_process_MAD(data_factors,columns_exclude=columns_index+columns_industry)
#     #缺失值处理
#     data_factors_2 = null_process(data_factors_1,columns_exclude=columns_index+columns_industry)
#     #市值行业中性
#     data_factors_3 = data_scale_neutral(data_factors_2,columns_index=columns_index,columns_industry=columns_industry,col_size=col_size)
#     #再次异常值处理
#     data_factors_3 = extreme_process_MAD(data_factors_3,columns_exclude=columns_index+columns_industry)
#     #标准化处理
#     data_factors_4 = data_scale_Z_Score(data_factors_3,columns_exclude=columns_index+columns_industry)
#     market_data=g.GetMarketData(stock_list,begT=dt, fields =["close"])
#     r_t = market_data[["close"]]
#     r_t1=g.GetMarketData(stock_list,begT=dt1, fields =["close"])[["close"]]
#     market_data["return"]=np.log(r_t1/r_t)
#     data_factors5=pd.merge(data_factors_4,market_data[["stock_id","return"]],on="stock_id",how="left")
#     df_train=pd.concat([df_train,data_factors5],axis=0)

In [1291]:
df_train_xgb=pd.DataFrame()
for i in range(len(dates)-1):
    dt = dates[i]
    if i==(len(dates)-1):
        dt1 ="2030-1-19"
    else:
        dt1 = dates[i+1]
    stock_list = g.stock_issueDate_filter(XNY_stock.stock_id,dt,delta=360)
    stock_list = stock_ST_filter(stock_list,dt)
    stock_list = stock_Suspended_filter(stock_list,dt)
    factors_quant=gd.GetFactorData(stocks=stock_list,begT=dt.replace("-",""), endT=0, fields=fields,type_id='')
    data_factors=pd.merge(factors_quant,industry_flag,on="stock_id",how="left")
    #异常值处理
    data_factors_1 = extreme_process_MAD(data_factors,columns_exclude=columns_index+columns_industry)
    #缺失值处理
    data_factors_2 = null_process(data_factors_1,columns_exclude=columns_index+columns_industry)
    market_data=g.GetMarketData(stock_list,begT=dt, fields =["close"])
    r_t = market_data[["close"]]
    r_t1=g.GetMarketData(stock_list,begT=dt1, fields =["close"])[["close"]]
    market_data["return"]=np.log(r_t1/r_t)
    data_factors5=pd.merge(data_factors_2,market_data[["stock_id","return"]],on="stock_id",how="left")
    df_train_xgb=pd.concat([df_train_xgb,data_factors5],axis=0)

In [1292]:
df_train_xgb.to_csv("P:\\Documents\\量化\\行业内因子选股\\新能源\\factor_data\\XNY_data_xgb.csv",index=False)

In [479]:
df_train.to_csv("P:\\Documents\\量化\\行业内因子选股\\新能源\\factor_data\\XNY_data.csv",index=False)

In [1079]:
df_train1=pd.read_csv("P:\\Documents\\量化\\行业内因子选股\\新能源\\factor_data\\XNY_data.csv",index_col=[1,0],encoding="utf-8")

In [1293]:
df_train_xgb2 = pd.read_csv("P:\\Documents\\量化\\行业内因子选股\\新能源\\factor_data\\XNY_data_xgb.csv",index_col=[1,0],encoding="utf-8")

In [1294]:
df_train_final_xgb = df_train_xgb2.fillna(0)

In [714]:
df_train1.shape

(144806, 96)

In [1082]:
# df_train2 = null_process(df_train1,columns_exclude=columns_index+columns_industry)

In [715]:
col_return=["return"]
#columns_industry

In [716]:
#计算RankIC
from scipy import stats
def get_RankIC(datas,col_return,columns_industry):
    factors_name=[i for i in datas.columns.tolist() if i not in (col_return+columns_industry)] #得到因子名
    ic = datas.groupby(level=0).apply(lambda data: [stats.spearmanr(data[factor],data["return"])[0] for factor in factors_name]) #得到的是以列表为值的序列
    ic = pd.DataFrame(ic.tolist(), index=ic.index, columns=factors_name) #得到各因子IC值,一个list为一个列
    return ic

In [717]:
#计算IC
from scipy import stats
def get_IC(datas,col_return,columns_industry):
    factors_name=[i for i in datas.columns.tolist() if i not in (col_return+columns_industry)] #得到因子名
    ic = datas.groupby(level=0).apply(lambda data: [stats.pearsonr(data[factor],data["return"])[0] for factor in factors_name]) #得到的是以列表为值的序列
    ic = pd.DataFrame(ic.tolist(), index=ic.index, columns=factors_name) #得到各因子IC值,一个list为一个列
    return ic

In [ ]:
#计算IR
# def IR_weight(datas):
#     datas_=datas.copy()
#     ic=get_ic(datas) #计算ic值，得到ic的
#     ic0 = ic.abs() #计算ic的绝对值
#     rolling_ic = ic0.rolling(52,min_periods=1).mean() #rolling为移动窗口函数,滚动12个月
#     rolling_ic_std=ic0.rolling(52,min_periods=1).std() #当滚动计算标准差时，起始日期得到的是缺失值，所以算完权重后，起始日期的值任用原值IC代替
#     IR=rolling_ic/rolling_ic_std #计算IR值
#     IR.iloc[0,:]=rolling_ic.iloc[0,:]
#     weight = IR.div(IR.sum(axis=1),axis=0) #计算IR权重,按行求和,按列相除
# return weight

In [1090]:
#IC
ic = get_RankIC(df_train1,col_return=col_return,columns_industry=columns_industry)

In [1091]:
#Rolling_IC
rolling_ic = ic.rolling(52).mean()

In [1092]:
#ICIR

In [1093]:
ICIR = (ic.rolling(52).mean()/ic.rolling(52).std())

In [585]:
# rolling_ic.abs().sort_values(by = "2022-07-08",ascending=False, axis=1)[rolling_ic.index=="2022-07-08"]

In [587]:
factor_growth = ["GROWTH_EarningRate_SQIncre",
"GROWTH_NetProfit_SQIncre_2Mcap",
"GROWTH_EstNetProfit_SQIncre_2Mcap",
"GROWTH_NPIncreQ2Q_2Mcap",
"GROWTH_NetProfit_SQYOY",
"GROWTH_NetProfit_Standard",
"GROWTH_ROE_SQYOY",
"GROWTH_ROE_TTMIncre",
"GROWTH_RROC",
"GROWTH_Acceleration",
"GROWTH_SalesEarning_SQYOY",
"GROWTH_Sales_SQYOY",
"GROWTH_Sales_SQIncre_2Mcap",
"GROWTH_SUE",
"GROWTH_DepositReceived_YOYIncre2Debt",
"GROWTH_Salary_YOYIncre2Ast",
"GROWTH_CON_NPCGRATE_1W",
"GROWTH_CON_NPCGRATE_4W",
"GROWTH_CON_NPCGRATE_13W"]

In [589]:
factor_profit = ["PROFIT_FCFF_5YHL",
"PROFIT_FCFF_5YHL-Std",
"PROFIT_FCFF_Stability",
"PROFIT_GrossProfitMargin_TTM",
"PROFIT_NetProfitMargin_TTM",
"PROFIT_ROE_TTM",
"PROFIT_ROE_SQ",
"PROFIT_ROE_Stability",
"PROFIT_ROIC_SQ",
"PROFIT_SalesEarningRatio_TTM",
"PROFIT_CON_NP_ROLLING",
"PROFIT_CON_OR_ROLLING",
"PROFIT_CON_NP"]

In [590]:
factor_quality = ["QUALITY_PledgeRatio",
"QUALITY_PublishDateGap",
"QUALITY_LiabilityRatio",
"QUALITY_CFO2Debt",
"QUALITY_CFO2Sales",
"QUALITY_CFO2NP",
"QUALITY_LiabilityRatio_YOYIncre"
]

In [591]:
factor_size =["SIZE_LnMarketValue"]

In [592]:
factor_qfii = ["QFII_HoldingPercent",
"QFII_PercentRise_15D",
"QFII_PercentRise_30D",
"QFII_PercentRise_60D"
]

In [593]:
factor_liquidity = ["LIQUIDITY_ILLIQ",
"LIQUIDITY_ILLIQ_10D",
"LIQUIDITY_TrunoverRate_1Mto3M",
"LIQUIDITY_TurnoverRate_Avg1M",
"LIQUIDITY_Volume_Std1M"
]

In [594]:
factor_reversal = ["REVERSAL_StockZF_10D",
"REVERSAL_StockZF_20D",
"REVERSAL_StockZF_40D",
"REVERSAL_StockZF_60D",
"REVERSAL_Extreme_Return",
"REVERSAL_MHigh_20D",
"REVERSAL_M_20D"]

In [595]:
factor_volatility =["VOLATILITY_RealizedVol_20D",
"VOLATILITY_DASTD",
"VOLATILITY_ATR20",
"VOLATILITY_M_20D",
"VOLATILITY_Amplitude_Mean"
]

In [596]:
factor_technical  = ["TECHNICAL_APB_1M",
"TECHNICAL_ARRP",
"TECHNICAL_Alpha_IR",
"TECHNICAL_Amplitude_Diff",
"TECHNICAL_Buy_to_Sell",
"TECHNICAL_BigOrderRatio_20D",
"TECHNICAL_BigOrderRatio_10D",
"TECHNICAL_BigOrderStrength_20D",
"TECHNICAL_BigOrderStrength_10D",
"TECHNICAL_DUVOL",
"TECHNICAL_DownwardVol",
"TECHNICAL_IVR",
"TECHNICAL_PVCorr_20D",
"TECHNICAL_Ratative_TurnVol",
"TECHNICAL_LongTermMom",
"TECHNICAL_Mild_Momentum",
"TECHNICAL_Quantile_Momentum",
"TECHNICAL_Quantile_TurnoverRate",
"TECHNICAL_QFII_MidTerm",
"TECHNICAL_Smart_Money"]

In [597]:
factor_emotion = ["EMOTION_MktConfidence_25D",
"EMOTION_MktConfidence_75D"]

In [724]:
#依据ICIR进行因子选择
def getBestFactor(ICIR,dt):
    f_top10 = list(ICIR.loc[[dt],].sort_values(by =dt,ascending=False, axis=1).columns[:10])
    f_growth_top5 = list(ICIR.loc[[dt],factor_growth].sort_values(by =dt,ascending=False, axis=1).columns[:5])
    f_profit_top5 = list(ICIR.loc[[dt],factor_profit].sort_values(by =dt,ascending=False, axis=1).columns[:5])
    f_quality_top3 = list(ICIR.loc[[dt],factor_quality].sort_values(by =dt,ascending=False, axis=1).columns[:3])
    #factor_size
    f_reversal_top3 = list(ICIR.loc[[dt],factor_reversal].sort_values(by =dt,ascending=False, axis=1).columns[:3])
    f_volatility_top3 = list(ICIR.loc[[dt],factor_volatility].sort_values(by =dt,ascending=False, axis=1).columns[:3])
    f_technical_top5 = list(ICIR.loc[[dt],factor_technical].sort_values(by =dt,ascending=False, axis=1).columns[:5])
    #factor_emotion
    f_selected =set(f_top10+f_growth_top5+f_profit_top5+f_reversal_top3+f_volatility_top3+f_technical_top5+factor_size+factor_emotion)
    return list(f_selected)

In [783]:
f_selected = getBestFactor(ICIR,dt)

### Backtest
#### Model update: monthly

In [1083]:
df_train_final = df_train1.fillna(0)

In [1096]:
df_backtest = pd.DataFrame()
var_selected_=OrderedDict()
model_backtest = OrderedDict()
df_all = df_train_final.copy()
df_temp = df_all.reset_index()
for i in range(36,(len(model_update_date)-1)):
    dt = model_update_date[i]
    dt_3yrs_ago = model_update_date[i-36]
    dt_1month_ahead = model_update_date[i+1]
    f_selected = getBestFactor(ICIR,dt)
    var_selected_[dt] = f_selected
    x_train = df_temp[(df_temp["date"]>dt_3yrs_ago)&(df_temp["date"]<=dt)].set_index(["date","stock_id"])[f_selected+columns_industry]
    y_train = df_temp[(df_temp["date"]>dt_3yrs_ago)&(df_temp["date"]<=dt)].set_index(["date","stock_id"])["return"]
    data_test =df_temp[(df_temp["date"]<=dt_1month_ahead)&(df_temp["date"]>dt)].set_index(["date","stock_id"])
    x_test = data_test[f_selected+columns_industry]
    model_lr = LinearRegression(fit_intercept = True)
    model_lr.fit(x_train,y_train)
    model_backtest[dt]=model_lr
    data_test["r_predict"]=model_lr.predict(x_test)
    df_backtest= pd.concat([df_backtest,data_test],axis=0)  

In [973]:
# r2_score(y_train,model_lr.predict(x_train))

In [974]:
# model_lr.feature_names_in_

In [887]:
import statsmodels.api as sm

In [894]:
# model = sm.OLS(np.asarray(y_train),np.asarray(x_train))

In [935]:
# x=sm.add_constant(x_train)

In [1125]:
def max_Drawdown(price_list):
    price_max= np.maximum.accumulate(price_list)
    drawdown_ratio=(price_max-price_list)/price_max
    max_drawdown=max(drawdown_ratio)
    return max_drawdown

### Trading Strategies

In [ ]:
#选出打分前100的股票
portfolio_stock_list = df_backtest[df_backtest["r_predict"].groupby(level=0).rank(ascending=False)<=100]

In [1228]:
#交易成本测算 - 单边千1.5
fee_est = portfolio_stock_list.reset_index()
turnover_rate = OrderedDict()
tran_fee=OrderedDict()
for i in range(len(portfolio_stock_list.index.levels[0])):
    date = portfolio_stock_list.index.levels[0][i]
    s_t1_list = fee_est[fee_est["date"]==date]["stock_id"]
    if i ==0:
        s_t0_list=list()
    else:
        date_t0=portfolio_stock_list.index.levels[0][i-1]
        s_t0_list = fee_est[fee_est["date"]==date_t0]["stock_id"]
    stock_change_pct=(len(s_t1_list)-s_t1_list.isin(list(s_t0_list)).sum())/len(s_t1_list)
    turnover_rate[date] = stock_change_pct
    if i==0:
        tran_cost = stock_change_pct*0.001
    else:
        tran_cost = stock_change_pct*0.002
    tran_fee[date] = tran_cost

In [1231]:
#基准交易成本测算 - 单边千1.5
fee_est = df_backtest.reset_index()
turnover_rate2 = OrderedDict()
tran_fee2=OrderedDict()
for i in range(len(portfolio_stock_list.index.levels[0])):
    date = portfolio_stock_list.index.levels[0][i]
    s_t1_list = fee_est[fee_est["date"]==date]["stock_id"]
    if i ==0:
        s_t0_list=list()
    else:
        date_t0=portfolio_stock_list.index.levels[0][i-1]
        s_t0_list = fee_est[fee_est["date"]==date_t0]["stock_id"]
    stock_change_pct=(len(s_t1_list)-s_t1_list.isin(list(s_t0_list)).sum())/len(s_t1_list)
    turnover_rate2[date] = stock_change_pct
    if i==0:
        tran_cost = stock_change_pct*0.001
    else:
        tran_cost = stock_change_pct*0.002
    tran_fee2[date] = tran_cost

In [1240]:
#等权买入选出股票，计算每期收益
portfolio_return = portfolio_stock_list["return"].groupby(level=0).mean()
#计算回测期平均收益，并转换为年化收益
print("组合回测期平均年化收益：")
print(portfolio_return.mean()*52)
print("组合回测期胜率：")
print((portfolio_return>0).sum()/len(portfolio_return))
print("组合回测期波动率：")
print((portfolio_return).std()*np.sqrt(52))
# print("组合回测期最大回测：")
# print(max_Drawdown(portfolio_return))
portfolio_return_wo_fee=(portfolio_return-list(tran_fee.values()))
print("组合回测期平均年化收益(除去交易成本）：")
print(portfolio_return_wo_fee.mean()*52)
turnover_rate_avg = np.average(list(turnover_rate.values()))
print("组合周度换仓平均换手率：")
print(turnover_rate_avg)

组合回测期平均年化收益：
0.1692811258696058
组合回测期胜率：
0.5871886120996441
组合回测期波动率：
0.28054023040887205
组合回测期平均年化收益(除去交易成本）：
0.14084212231088697
组合周度换仓平均换手率：
0.27523131672597867


In [1232]:
#等权买入所有股票，作为基准
benchmark_return = df_backtest["return"].groupby(level=0).mean()
print("基准回测期平均年化收益：")
print(benchmark_return.mean()*52)
print("基准回测期胜率：")
print((benchmark_return>0).sum()/len(benchmark_return))
print("基准回测期波动率：")
print((benchmark_return).std()*np.sqrt(52))
benchmark_return_wo_fee=(benchmark_return-list(tran_fee2.values()))
print("基准回测期平均年化收益(除去交易成本）：")
print(benchmark_return_wo_fee.mean()*52)

基准回测期平均年化收益：
0.06663579220870032
基准回测期胜率：
0.5338078291814946
基准回测期波动率：
0.2525838691415711
基准回测期平均年化收益(除去交易成本）：
0.0659111157299805


In [1305]:
np.average(list(turnover_rate2.values()))

0.008747402495218805

In [1233]:
#超额收益
active_return = (portfolio_return-benchmark_return)
print("超额收益回测期平均年化收益：")
print(active_return.mean()*52)
print("超额收益回测期胜率：")
print((active_return>0).sum()/len(active_return))
print("超额收益回测期波动率：")
print((active_return).std()*np.sqrt(52))
active_return_wo_fee = (portfolio_return_wo_fee-benchmark_return_wo_fee)
print("超额收益回测期平均年化收益(除去交易成本）：")
print(active_return_wo_fee.mean()*52)

超额收益回测期平均年化收益：
0.10264533366090542
超额收益回测期胜率：
0.6405693950177936
超额收益回测期波动率：
0.13267280468021184
超额收益回测期平均年化收益(除去交易成本）：
0.07493100658090646


In [1124]:
ic_strategy = df_backtest.groupby(level=0).apply(lambda data: stats.spearmanr(data["return"],data["r_predict"])[0])
rolling_ic_strategy = ic_strategy.rolling(52).mean()
print("IC of Predicted Return：")
print(ic_strategy.mean())

IC of Predicted Return：
0.07134377266735534


### Xgboost

In [1297]:
df_backtest2 = pd.DataFrame()
var_selected_2=OrderedDict()
# model_backtest2 = OrderedDict()
df_all2 = df_train_final_xgb.copy()
df_temp2 = df_all2.reset_index()
for i in range(36,(len(model_update_date)-1)):
    dt = model_update_date[i]
    dt_3yrs_ago = model_update_date[i-36]
    dt_1month_ahead = model_update_date[i+1]
    f_selected = getBestFactor(ICIR,dt)
    var_selected_2[dt] = f_selected
    x_train2 = df_temp2[(df_temp2["date"]>dt_3yrs_ago)&(df_temp2["date"]<=dt)].set_index(["date","stock_id"])[f_selected+columns_industry]
    y_train2 = df_temp2[(df_temp2["date"]>dt_3yrs_ago)&(df_temp2["date"]<=dt)].set_index(["date","stock_id"])["return"]
    data_test2 =df_temp2[(df_temp2["date"]<=dt_1month_ahead)&(df_temp2["date"]>dt)].set_index(["date","stock_id"])
    x_test2 = data_test2[f_selected+columns_industry]
    model_xgb = xgb.XGBRegressor(silent=True,learning_rate =0.05,n_estimators=1000,max_depth=7,subsample=1,colsample_bytree=1,seed=0,min_child_weight=10,nthread=-1)
    model_xgb.fit(x_train2, y_train2)
#     model_backtest2[dt]=model_xgb
    data_test2["r_predict"]=model_xgb.predict(x_test2)
    df_backtest2= pd.concat([df_backtest2,data_test2],axis=0)  

[21:40:07] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:40:16] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:40:25] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then bein

In [1299]:
portfolio_stock_list2 = df_backtest2[df_backtest2["r_predict"].groupby(level=0).rank(ascending=False)<=100]

In [1300]:
#交易成本测算 - 单边千1
fee_est3 = portfolio_stock_list2.reset_index()
turnover_rate3 = OrderedDict()
tran_fee3=OrderedDict()
for i in range(len(portfolio_stock_list2.index.levels[0])):
    date = portfolio_stock_list2.index.levels[0][i]
    s_t1_list3 = fee_est3[fee_est3["date"]==date]["stock_id"]
    if i ==0:
        s_t0_list3=list()
    else:
        date_t0=portfolio_stock_list2.index.levels[0][i-1]
        s_t0_list3 = fee_est3[fee_est3["date"]==date_t0]["stock_id"]
    stock_change_pct3=(len(s_t1_list3)-s_t1_list3.isin(list(s_t0_list3)).sum())/len(s_t1_list3)
    turnover_rate3[date] = stock_change_pct3
    if i==0:
        tran_cost = stock_change_pct3*0.001
    else:
        tran_cost = stock_change_pct3*0.002
    tran_fee3[date] = tran_cost

In [1268]:
#xgboost
#等权买入选出股票，计算每期收益
portfolio_return2 = portfolio_stock_list2["return"].groupby(level=0).mean()
#计算回测期平均收益，并转换为年化收益
print("组合回测期平均年化收益：")
print(portfolio_return2.mean()*52)
print("组合回测期胜率：")
print((portfolio_return2>0).sum()/len(portfolio_return2))
print("组合回测期波动率：")
print((portfolio_return2).std()*np.sqrt(52))
# print("组合回测期最大回测：")
# print(max_Drawdown(portfolio_return2))
portfolio_return_wo_fee2=(portfolio_return2-list(tran_fee3.values()))
print("组合回测期平均年化收益(除去交易成本）：")
print(portfolio_return_wo_fee2.mean()*52)
turnover_rate_avg3 = np.average(list(turnover_rate3.values()))
print("组合周度换仓平均换手率：")
print(turnover_rate_avg3)

组合回测期平均年化收益：
0.13678979780959682
组合回测期胜率：
0.5551601423487544
组合回测期波动率：
0.2602026011880928
组合回测期平均年化收益(除去交易成本）：
0.08399406827223029
组合周度换仓平均换手率：
0.5094306049822063


In [1301]:
#xgboost
#等权买入选出股票，计算每期收益
portfolio_return2 = portfolio_stock_list2["return"].groupby(level=0).mean()
#计算回测期平均收益，并转换为年化收益
print("组合回测期平均年化收益：")
print(portfolio_return2.mean()*52)
print("组合回测期胜率：")
print((portfolio_return2>0).sum()/len(portfolio_return2))
print("组合回测期波动率：")
print((portfolio_return2).std()*np.sqrt(52))
# print("组合回测期最大回测：")
# print(max_Drawdown(portfolio_return2))
portfolio_return_wo_fee2=(portfolio_return2-list(tran_fee3.values()))
print("组合回测期平均年化收益(除去交易成本）：")
print(portfolio_return_wo_fee2.mean()*52)
turnover_rate_avg3 = np.average(list(turnover_rate3.values()))
print("组合周度换仓平均换手率：")
print(turnover_rate_avg3)

组合回测期平均年化收益：
0.12109774487856266
组合回测期胜率：
0.5444839857651246
组合回测期波动率：
0.2641448633264826
组合回测期平均年化收益(除去交易成本）：
0.07571525377535983
组合周度换仓平均换手率：
0.4381494661921709


In [1271]:
#等权买入所有股票，作为基准
benchmark_return = df_backtest["return"].groupby(level=0).mean()
print("基准回测期平均年化收益：")
print(benchmark_return.mean()*52)
print("基准回测期胜率：")
print((benchmark_return>0).sum()/len(benchmark_return))
print("基准回测期波动率：")
print((benchmark_return).std()*np.sqrt(52))
benchmark_return_wo_fee=(benchmark_return-list(tran_fee2.values()))
print("基准回测期平均年化收益(除去交易成本）：")
print(benchmark_return_wo_fee.mean()*52)

基准回测期平均年化收益：
0.06663579220870032
基准回测期胜率：
0.5338078291814946
基准回测期波动率：
0.2525838691415711
基准回测期平均年化收益(除去交易成本）：
0.0659111157299805


In [1269]:
#等权买入所有股票，作为基准
active_return2 = (portfolio_return2-benchmark_return)
print("超额收益回测期平均年化收益：")
print(active_return2.mean()*52)
print("超额收益回测期胜率：")
print((active_return2>0).sum()/len(active_return2))
print("超额收益回测期波动率：")
print((active_return2).std()*np.sqrt(52))
active_return_wo_fee2 = (portfolio_return_wo_fee2-benchmark_return_wo_fee)
print("超额收益回测期平均年化收益(除去交易成本）：")
print(active_return_wo_fee2.mean()*52)

超额收益回测期平均年化收益：
0.07015400560089657
超额收益回测期胜率：
0.6512455516014235
超额收益回测期波动率：
0.0481481308013867
超额收益回测期平均年化收益(除去交易成本）：
0.018082952542249835
